In this assignment, I used randomly choosed 2_dimensional and 3_dimensional points for calculating projection matrix(P). Then, I used ORB(Oriented FAST and Rotated BRIEF) for feature matching. Using these new knowledges, I calculated world coordinates for the given points in the reference image. Using the knowledge of the coordinates, I will be able to find Rotational Matrix(R) and translation matrix(T). But I couldn't take the inverse of the matrix. I couldn't find these informations. 

In [ ]:
import numpy as np
data_2d = np.load('vr2d.npy')
data_3d = np.load('vr3d.npy')

In [2]:
row1 = np.array([[data_2d[0][0][0], data_2d[0][0][1], 1]], np.float32)
row2 = np.array([[data_2d[4][0][0], data_2d[4][0][1], 1]], np.float32)
row3 = np.array([[data_2d[6][0][0], data_2d[6][0][1], 1]], np.float32)
row4 = np.array([[data_2d[12][0][0], data_2d[6][0][1], 1]], np.float32)

z_c_1 = data_3d[0][0][2]
z_c_2 = data_3d[4][0][2]
z_c_3 = data_3d[6][0][2]
z_c_4 = data_3d[12][0][2]

z1 = data_3d[0][0]
z2 = data_3d[4][0]
z3 = data_3d[6][0]
z4 = data_3d[12][0]

one_vector = np.array([[1]],np.float32)
z1 = np.append(z1,one_vector)
z2 = np.append(z2,one_vector)
z3 = np.append(z3,one_vector)
z4 = np.append(z4,one_vector)

In [3]:
Q  =np.column_stack((z1,z2,z3,z4))
Q

array([[  4.076994 ,   2.5796685,   1.9781952,   8.264166 ],
       [-10.0050955,  -0.9734893,  -2.4412844,  -3.2083879],
       [  2.4227831,   1.8305027,   1.4702834,   4.441536 ],
       [  1.       ,   1.       ,   1.       ,   1.       ]],
      dtype=float32)

In [4]:
row1 = np.multiply(row1,z_c_1)
row2 = np.multiply(row2,z_c_2)
row3 = np.multiply(row3,z_c_3)
row4 = np.multiply(row4,z_c_4)

In [5]:
col1 = np.transpose(row1)
col2 = np.transpose(row2)
col3 = np.transpose(row3)
col4 = np.transpose(row4)

In [6]:
A = np.concatenate((col1,col2,col3,col4),axis=1)
A

array([[3.29013940e+03, 1.05803064e+03, 1.02331726e+03, 3.81972070e+03],
       [1.29376624e+03, 1.08365759e+03, 9.35100220e+02, 2.82481689e+03],
       [2.42278314e+00, 1.83050275e+00, 1.47028339e+00, 4.44153595e+00]],
      dtype=float32)

In [7]:
Inv = np.linalg.inv(Q)
Inv

array([[ -1.4940624 ,  -0.15373863,   3.1211414 ,  -2.008735  ],
       [ -7.0050197 ,  -0.08011552,  14.799109  ,  -8.097174  ],
       [  7.1708755 ,   0.1748571 , -15.462091  ,   9.975142  ],
       [  1.3282068 ,   0.05899706,  -2.4581592 ,   1.1307671 ]],
      dtype=float32)

In [8]:
P = np.matmul(A,Inv)
P

array([[ 8.4260498e+01, -1.8629961e+02,  7.1479492e+02, -6.4912671e+02],
       [ 9.3341748e+02,  4.4445171e+01, -1.3272607e+03,  1.1485698e+03],
       [ 0.0000000e+00,  2.2351742e-08,  1.0000010e+00,  0.0000000e+00]],
      dtype=float32)

In [9]:
z_test = data_3d[5][0]
one_vector = np.array([[1]],np.float32)
Z_test = np.append(z_test,one_vector)
np.matmul(P,Z_test)

array([1055.7664   , 1085.8562   ,    1.8258733], dtype=float32)

In [10]:
z_test

array([ 2.5757904, -0.9808507,  1.8258715], dtype=float32)

In [11]:
data_2d[5][0]

array([578., 592.], dtype=float32)

In [12]:
1055.7664/1.8258733

578.2254442298926

In [13]:
1085.8562/1.8258733

594.7051200102438

In [14]:
import cv2

img1 = cv2.imread('img1.png')
img2 = cv2.imread('img2.png')

orb = cv2.ORB_create()

keypoints1, descriptors1 = orb.detectAndCompute(img1, None)
keypoints2, descriptors2 = orb.detectAndCompute(img2,None)

bf = cv2.BFMatcher_create(cv2.NORM_HAMMING,crossCheck=True)
matches = bf.match(descriptors1, descriptors2)

matches = sorted(matches,key=lambda x:x.distance)    

ORB_matches =cv2.drawMatches(img1, keypoints1, img2, keypoints2, matches[:4], None, flags=2)
cv2.imwrite('resulted.png',ORB_matches)

best_matches = matches[:4]
print(best_matches)

[<DMatch 0000022A4333DE90>, <DMatch 0000022A4333DD50>, <DMatch 0000022A4334E410>, <DMatch 0000022A4333D310>]


In [15]:
import numpy

world_coordinates= np.array([], dtype=np.float32)
list_of_coordinates=[]
for mat in best_matches:

    # Get the matching keypoints for each of the images
    img1_idx = mat.queryIdx
    img2_idx = mat.trainIdx

    # x - columns
    # y - rows
    (x1,y1) = keypoints1[img1_idx].pt
    (x2,y2) = keypoints2[img2_idx].pt
    
    image1_coordinates = np.array([[x1, y1, 1]], np.float32)
    image1_coordinates = np.transpose(image1_coordinates)
    last_column = P[:,-1]
    last_column = last_column.reshape(-1,1)
    updated_result = np.subtract(image1_coordinates,last_column)
    P_last_column_removed= P[:, :-1]
    P_New_Inv = np.linalg.inv(P_last_column_removed)
    
    three_dimension_world_coordinates = np.matmul(P_New_Inv,updated_result)
    list_of_coordinates.append(three_dimension_world_coordinates)

world_coordinates  =np.column_stack((list_of_coordinates[0],list_of_coordinates[1],list_of_coordinates[2],list_of_coordinates[3]))
world_coordinates

array([[ 0.53359276,  0.8010581 ,  0.7988345 ,  0.79745305],
       [-3.5467124 , -8.059723  , -8.053458  , -8.272367  ],
       [ 0.9999991 ,  0.9999992 ,  0.9999992 ,  0.9999992 ]],
      dtype=float32)

In [16]:
import cv2
import numpy

img1 = cv2.imread('img1.png')
img3 = cv2.imread('img3.png')

orb = cv2.ORB_create()

keypoints1, descriptors1 = orb.detectAndCompute(img1, None)
keypoints3, descriptors3 = orb.detectAndCompute(img3,None)

bf_2 = cv2.BFMatcher_create(cv2.NORM_HAMMING,crossCheck=True)
matches_2 = bf.match(descriptors1, descriptors3)

matches_2 = sorted(matches_2,key=lambda x:x.distance)    

ORB_matches_2 =cv2.drawMatches(img1, keypoints1, img3, keypoints3, matches[:4], None, flags=2)

cv2.imwrite('resulted2.png',ORB_matches_2)
best_matches_2 = matches_2[:4]
print(best_matches_2)

[<DMatch 0000022A4334FB10>, <DMatch 0000022A43358270>, <DMatch 0000022A4334FBF0>, <DMatch 0000022A4334E7D0>]


In [17]:
world_coordinates_2= np.array([], dtype=np.float32)
list_of_coordinates_2=[]
for mat2 in best_matches_2:

    # Get the matching keypoints for each of the images
    img1_idx = mat2.queryIdx
    img2_idx = mat2.trainIdx

    # x - columns
    # y - rows
    (x1,y1) = keypoints1[img1_idx].pt
    (x2,y2) = keypoints2[img2_idx].pt


    image1_marked = cv2.circle(img1, (int(x1),int(y1)), 5, (0,0,255), 1)   
    image3_marked = cv2.circle(img2, (int(x2),int(y2)), 5, (0,0,255), 1) 
    
    image1_coordinates = np.array([[x1, y1, 1]], np.float32)
    image1_coordinates = np.transpose(image1_coordinates)
    last_column = P[:,-1]
    last_column = last_column.reshape(-1,1)
    updated_result = np.subtract(image1_coordinates,last_column)
    P_last_column_removed= P[:, :-1]
    P_New_Inv = np.linalg.inv(P_last_column_removed)
    
    three_dimension_world_coordinates = np.matmul(P_New_Inv,updated_result)
    list_of_coordinates_2.append(three_dimension_world_coordinates)

world_coordinates_2  =np.column_stack((list_of_coordinates_2[0],list_of_coordinates_2[1],list_of_coordinates_2[2],list_of_coordinates_2[3]))
world_coordinates_2

array([[ 0.7926368 ,  0.8124252 ,  0.49886245,  0.6679777 ],
       [-8.03622   , -8.2717    , -5.2576504 , -8.545634  ],
       [ 0.9999992 ,  0.9999992 ,  0.99999917,  0.9999992 ]],
      dtype=float32)